In [82]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [83]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[QQ(q).numerator(),QQ(q).denominator()] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R = PolynomialRing(K, 'x')
        x = R.gen()
        if (x**2 - n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [84]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [85]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [86]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [87]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def dft(SGA,unitary=False):
    K = containing_field(SGA)
    fourier_transform = [flatten([hat(delta(g),partition,SGA,K,unitary).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]
    if unitary:
        return matrix(K,fourier_transform).transpose()
    else:
        return matrix(fourier_transform).transpose()

In [88]:
n = 4

In [89]:
SGA = SymmetricGroupAlgebra(QQbar,n)

In [90]:
K = containing_field(SGA); K

Algebraic Field

In [91]:
G = SGA.group()

In [92]:
SGA_dft = SGA.dft(); SGA_dft

24 x 24 dense matrix over Algebraic Field (use the '.str()' method to see the entries)

In [93]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

24 x 24 dense matrix over Algebraic Field (use the '.str()' method to see the entries)

In [94]:
partition = Partitions(SGA.group().degree())[1]; partition

[3, 1]

In [95]:
specht_module = SGA.specht_module(partition)

In [96]:
rho = specht_module.representation_matrix

In [97]:
group_size = SGA.group().cardinality()

In [98]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Algebraic Field

In [99]:
Q = unitary_change_of_basis(SGA,partition,K); Q

[ 1.154700538379252? 0.2886751345948129? 0.2886751345948129?]
[0.2886751345948129?  1.154700538379252? 0.2886751345948129?]
[0.2886751345948129? 0.2886751345948129?  1.154700538379252?]

In [100]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/8

In [101]:
K

Algebraic Field

In [102]:
G[3]

[1, 3, 4, 2]

In [103]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[           0.?e-15 1.000000000000000?            0.?e-15]
[           0.?e-15            0.?e-15 1.000000000000000?]
[1.000000000000000?            0.?e-16            0.?e-16]

In [104]:
U_rho*U_rho.conjugate().transpose()

[1.000000000000000?            0.?e-15            0.?e-15]
[           0.?e-15 1.000000000000000?            0.?e-15]
[           0.?e-15            0.?e-15 1.000000000000000?]

In [105]:
U_dft = dft(SGA,unitary=True); U_dft.base_ring()

Algebraic Field

In [106]:
U_dft*U_dft.conjugate().transpose()

24 x 24 dense matrix over Algebraic Field (use the '.str()' method to see the entries)

In [107]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [108]:
#QUESTION: what are the eigenvalues?
#for n=3, the minimal polynomial is degree 24 for the eigenvalues. the splitting field is degree 192.
#this means that the Galois group is nonabelian, and since the minimal polynomial has rational coefficients
#the eigenvalues are not roots of unity

In [109]:
U_dft.charpoly()

x^24 + 0.?e2*x^23 - 0.8863652921131?*x^22 - 0.14427746470?*x^21 - 0.169573238?*x^20 + 0.47102068?*x^19 + 0.2676418?*x^18 - 0.4186792?*x^17 - 0.025348?*x^16 + 0.048954?*x^15 - 0.245604?*x^14 + 0.257632?*x^13 - 0.2576317342623?*x^11 + 0.2456032429?*x^10 - 0.048953670?*x^9 + 0.02534808?*x^8 + 0.4186792?*x^7 - 0.2676417?*x^6 - 0.471021?*x^5 + 0.169574?*x^4 + 0.144278?*x^3 + 0.886366?*x^2 + 0.1725460?*x - 1.0000000?

In [110]:
#ISSUE: the splitting field appears to be high degree, and the coefficients are large
#the defining polynomial is monic with integer coefficients
#since the eigenvalues lie on the unit circle, a result of Kroenecker implies they are roots of unity
if len(U_dft.charpoly().factor()) != U_dft.charpoly().degree():
    L.<a> = U_dft.charpoly().splitting_field(); L
else:
    L = K

In [111]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
if L != L.algebraic_closure():
    L_deg = L.absolute_degree()
    print(L_deg)

In [112]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
if L != L.algebraic_closure():
    L_G = L.galois_group()

In [113]:
#one can factor the polynomial over a splitting field L/K
#but there is no way to express the roots of a quintic in terms of radicals
try:
    eigs = matrix(L,U_dft).eigenvalues(extend=False)
except TypeError:
    print("Cannot express eigenvalues in terms of radicals since polynomial is a quintic or above.")

In [149]:
eigs

[-1, 1, -0.9824456136768473? - 0.1865492325557063?*I, -0.9824456136768473? + 0.1865492325557063?*I, -0.8985365719703156? - 0.4388986543973836?*I, -0.8985365719703156? + 0.4388986543973836?*I, -0.7578648392050981? - 0.6524115920924694?*I, -0.7578648392050981? + 0.6524115920924694?*I, -0.4633311957343842? - 0.8861851968179935?*I, -0.4633311957343842? + 0.8861851968179935?*I, -0.2805792598087796? - 0.9598308595607651?*I, -0.2805792598087796? + 0.9598308595607651?*I, -0.03584488349997613? - 0.9993573656739981?*I, -0.03584488349997613? + 0.9993573656739981?*I, 0.2574814524105171? - 0.9662832409105369?*I, 0.2574814524105171? + 0.9662832409105369?*I, 0.5606995539799094? - 0.8280193295852039?*I, 0.5606995539799094? + 0.8280193295852039?*I, 0.7835078863685053? - 0.6213818407375271?*I, 0.7835078863685053? + 0.6213818407375271?*I, 0.9104280922597108? - 0.4136673649495976?*I, 0.9104280922597108? + 0.4136673649495976?*I, 0.9927583939109320? - 0.1201281454089212?*I, 0.9927583939109320? + 0.120128145

In [114]:
[arg(eig).n(20) for eig in eigs]

[3.1416,
 0.00000,
 -2.9539,
 2.9539,
 -2.6872,
 2.6872,
 -2.4308,
 2.4308,
 -2.0525,
 2.0525,
 -1.8552,
 1.8552,
 -1.6066,
 1.6066,
 -1.3104,
 1.3104,
 -0.97557,
 0.97557,
 -0.67051,
 0.67051,
 -0.42648,
 0.42648,
 -0.12042,
 0.12042]

In [147]:
eigs[0].minpoly()

x + 1

In [115]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [116]:
SGA_dft.base_ring()

Algebraic Field

In [117]:
SGA_eigs = SGA_dft.eigenvalues(); SGA_eigs

[3.199568489272891? + 0.2514410409879547?*I, 3.199568489272891? - 0.2514410409879547?*I, 2.993228869092294? + 0.912124753071154?*I, 2.993228869092294? - 0.912124753071154?*I, 2.642899281891904? + 1.590091131310560?*I, 2.642899281891904? - 1.590091131310560?*I, 2.188474879179236? + 1.931582580658714?*I, 2.188474879179236? - 1.931582580658714?*I, 1.486470426769980? + 2.482244638968972?*I, 1.486470426769980? - 2.482244638968972?*I, 0.7700453938741760? + 2.977686796570087?*I, 0.7700453938741760? - 2.977686796570087?*I, -0.012076105575176902? + 3.280400303454078?*I, -0.012076105575176902? - 3.280400303454078?*I, -0.804952639345548? + 2.912731604176754?*I, -0.804952639345548? - 2.912731604176754?*I, -2.051213395756673? + 2.286358575987839?*I, -2.051213395756673? - 2.286358575987839?*I, -2.745916288430809? + 0.997466441758705?*I, -2.745916288430809? - 0.997466441758705?*I, -2.786686565676190? + 1.302381740159278?*I, -2.786686565676190? - 1.302381740159278?*I, -3.032620123073862? + 0.610585810

In [118]:
[abs(eig) for eig in SGA_eigs]

[3.209433145376473?,
 3.209433145376473?,
 3.129119784848871?,
 3.129119784848871?,
 3.084364832521801?,
 3.084364832521801?,
 2.918978170987708?,
 2.918978170987708?,
 2.893290994240283?,
 2.893290994240283?,
 3.075644414930741?,
 3.075644414930741?,
 3.280422531203484?,
 3.280422531203484?,
 3.021912333202181?,
 3.021912333202181?,
 3.071630175155329?,
 3.071630175155329?,
 2.921471472649425?,
 2.921471472649425?,
 3.076007186665932?,
 3.076007186665932?,
 3.093476982702490?,
 3.093476982702490?]

In [119]:
SGA_eigs[0].minpoly()

x^24 - 133/36*x^23 - 1423/216*x^22 + 61015/1296*x^21 - 2504/27*x^20 - 1138789/23328*x^19 + 4168451/2592*x^18 - 329337989/69984*x^17 - 317117995/104976*x^16 + 33502427/972*x^15 - 1239088667/26244*x^14 - 928700101/6561*x^13 + 5176853816/6561*x^12 - 2596344938/6561*x^11 - 37779941960/6561*x^10 + 40835832032/2187*x^9 - 30574432000/2187*x^8 - 41981126656/243*x^7 + 247198662656/243*x^6 - 190156832768/81*x^5 - 95325519872/27*x^4 + 33726398464*x^3 - 43318771712*x^2 - 157840048128*x + 463856467968

In [120]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((SGA_dft*SGA_dft.transpose()).numpy().diagonal()))

[4.89897949+0.j 4.89897949+0.j 4.        +0.j 3.46410162+0.j
 3.46410162+0.j 3.46410162+0.j 3.46410162+0.j 3.26598632+0.j
 3.26598632+0.j 3.        +0.j 3.        +0.j 3.        +0.j
 2.82842712+0.j 2.82842712+0.j 2.82842712+0.j 2.82842712+0.j
 2.82842712+0.j 2.82842712+0.j 2.66666667+0.j 2.66666667+0.j
 2.44948974+0.j 2.44948974+0.j 2.30940108+0.j 2.30940108+0.j]
[4.898979485566356? 2.828427124746190? 2.449489742783178?
 2.309401076758503? 3.265986323710904? 2.828427124746190? 8/3
 3.464101615137755? 3 2.828427124746190? 3.464101615137755? 3 4
 3.464101615137755? 2.828427124746190? 8/3 2.309401076758503? 3
 2.828427124746190? 2.449489742783178? 3.464101615137755?
 3.265986323710904? 2.828427124746190? 4.898979485566356?]
